## Libraries & Constants

In [1]:
import requests
from bs4 import BeautifulSoup
import csv

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import json

In [2]:
BASE_URL = "http://pogoda-service.ru/archive_gsod.php?"
country = "RS"

## Station ID's receiver
Since the website updates using JavaScript, I have to simulate a real browser using Selenium, to pick the country in a form and scrape all station id's for this station in the next box.

In [ ]:
# Set up the Selenium WebDriver (ensure you have ChromeDriver installed)
driver = webdriver.Chrome()

# Open the website
driver.get(BASE_URL)

# Select Russia from the country dropdown
country_dropdown = Select(driver.find_element(By.NAME, "country"))
# support for finding the weather in any country
country_map = {
    option.get_attribute("value"): option.text.strip()
    for option in country_dropdown.options
    if option.get_attribute("value")
}
print(f"Found {len(country_map)} countries")

country_dropdown.select_by_value(country)  # Assuming 'RS' is the correct value
# Wait for the page to update (adjust sleep time as needed)
time.sleep(2)

# Get the station dropdown after it updates
station_dropdown = Select(driver.find_element(By.NAME, "station"))
stations = {
    option.get_attribute("value"): {
        "name": option.text.strip(),
        "country_code": country,
        "country_name": country_map[country]
    }
    for option in station_dropdown.options
    if option.get_attribute("value")
}

print(f"Total station count in {country}: {len(stations)}")
# --- Save to JSON
with open("station_metadata.json", "w", encoding="utf-8") as f:
    json.dump(stations, f, ensure_ascii=False, indent=2)

# Close the browser
driver.quit() 

Found 249 countries
Total station count in RS: 2082


## Actual data retrieval
All data can be received without the annoying form, since the link itself is the query. It looks like this: http://pogoda-service.ru/archive_gsod_res.php?country=RS&station=id&datepicker_beg=dd.mm.yyyy&datepicker_end=dd.mm.yyyy, so all I need to do is go through all such links. The dataset seems to be rarely updated, so some less busy/famous stations have missing data, but it will do.

In [ ]:
import requests
import csv
import time
import os
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm

BASE_URL = "http://pogoda-service.ru/archive_gsod_res.php"
#station_ids = ["287854", "294850", "223650", "260780"]  # Example station IDs for testing, replace with real ones
station_ids = stations

# Define date range (min 1934)
start_year = 2014
end_year = 2024

def fetch_weather_data(country, station_id, start_date, end_date):
    """Fetch weather data for a station between two dates."""
    params = {
        "country": country,
        "station": station_id,
        "datepicker_beg": start_date.strftime("%d.%m.%Y"),
        "datepicker_end": end_date.strftime("%d.%m.%Y"),
    }
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", class_="table_res")
        
        if not table:
            tqdm.write(f"No data found for station {station_id} from {start_date} to {end_date}")
            return []

        # Extract table rows
        rows = table.find_all("tr")[1:]  # Skip header row
        data = []
        
        for row in rows:
            cols = [td.text.strip() for td in row.find_all("td")]
            if len(cols) == 8:
                data.append(cols)
        
        return data
    else:
        tqdm.write(f"Failed to fetch data for station {station_id}: {response.status_code}")
        return []

def save_to_csv(station_id, data):
    """Save weather data to a CSV file."""
    filename = f"stations/station_{station_id}.csv"
    headers = ["date", "min_temp", "max_temp", "avg_temp", "atm_pressure", "wind_speed", "downfall", "eff_temp"]
    
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(data)
    
    tqdm.write(f"Saved data for station {station_id} to {filename}")
    
def check_csv(station_id):
    filename = f"stations/station_{station_id}.csv"
    return os.path.exists(filename)

def scrape_weather_data(country, station_ids):
    """Main function to loop through all stations and fetch/save their data."""
    
    for station_id in tqdm(station_ids, desc="Scraping Stations", unit="station"):
        
        filename = f"station_{station_id}.csv"

        # Skip if CSV already exists
        if os.path.exists(filename):
            tqdm.write(f"Skipping station {station_id}, CSV already exists.")
            continue  
        
        all_data = []
        current_date = datetime(start_year, 1, 1)
        end_date = datetime(end_year, 12, 31)
        
        date_range = []
        while current_date < end_date:
            next_date = min(current_date + timedelta(days=1000), end_date)  # Page can display 1000 items at a time
            date_range.append((current_date, next_date))
            current_date = next_date + timedelta(days=1)  # Move to next chunk

        # Progress bar for date ranges
        for start_date, next_date in tqdm(date_range, desc=f"Fetching {station_id}", unit="year", leave=False):
            data = fetch_weather_data(country, station_id, start_date, next_date)
            all_data.extend(data)
            time.sleep(0.5)  # avoid spam
        
        if all_data:
            save_to_csv(station_id, all_data)
        else:
            tqdm.write("No records in a given period.")

# Run the scraping
scrape_weather_data(country, station_ids)


Scraping Stations:   0%|          | 0/2082 [00:00<?, ?station/s]

Skipping station 287854, CSV already exists.


Scraping Stations:   0%|          | 2/2082 [00:12<3:43:09,  6.44s/station]

No records in a given period.


Scraping Stations:   0%|          | 3/2082 [00:23<4:42:27,  8.15s/station]

...

No records in a given period.


Scraping Stations:  26%|██▋       | 549/2082 [1:46:24<5:18:02, 12.45s/station]

No records in a given period.


Scraping Stations:  26%|██▋       | 549/2082 [1:46:45<4:58:07, 11.67s/station]


ConnectionError: HTTPConnectionPool(host='pogoda-service.ru', port=80): Max retries exceeded with url: /archive_gsod_res.php?country=RS&station=360380&datepicker_beg=09.01.2024&datepicker_end=31.12.2024 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020D390429E0>: Failed to establish a new connection: [WinError 10053] Программа на вашем хост-компьютере разорвала установленное подключение'))

In [10]:
check_csv(200870)

True